In [ ]:

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re

from math import exp
from numpy import sign

from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
DEPRES_NROWS = 516088  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 683702 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 90 # Max tweet size
MAX_NB_WORDS = 1200000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

In [ ]:
sdf=pd.read_csv('/content/gdrive/My Drive/Capstone/dataset/final_dataset.csv')

In [ ]:
sdf=sdf.drop(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'created_at', 'tweet',	'language',	'clean_tweet',	'clean_tweet_exp',	'cleanest_tweet','user_id'], axis = 1 )
sdf = sdf.sample(frac = 1)
sdf['lem_tweet']=sdf['lem_tweet'].map(str)
tweets=sdf['lem_tweet']



In [ ]:
depressive_tweets_arr=sdf.loc[sdf['vader_sentiment_label'] == 0.0]
depressive_tweets_arr=depressive_tweets_arr.drop(['vader_score','vader_sentiment_label'],axis = 1)
depressive_tweets_arr.head(5)
depressive_tweets_arr=list(depressive_tweets_arr['lem_tweet'])
X_d=depressive_tweets_arr
depressive_tweets_arr[0]
len(depressive_tweets_arr)


516088

In [ ]:
random_tweets_arr=sdf.loc[sdf['vader_sentiment_label'] == 1.0]
random_tweets_arr=random_tweets_arr.drop(['vader_score','vader_sentiment_label'],axis = 1)
random_tweets_arr.head(5)
random_tweets_arr=list(random_tweets_arr['lem_tweet'])
random_tweets_arr[0]
X_r=random_tweets_arr
len(random_tweets_arr)

683702

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Capstone/dataset/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_d + X_r)


In [ ]:
sequences_d = tokenizer.texts_to_sequences(X_d)
sequences_r = tokenizer.texts_to_sequences(X_r)
print(sequences_d)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 380536 unique tokens


In [ ]:

data_d = pad_sequences(sequences_d, maxlen=MAX_SEQUENCE_LENGTH)
data_r = pad_sequences(sequences_r, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_d tensor:', data_d.shape)
print('Shape of data_r tensor:', data_r.shape)


Shape of data_d tensor: (516088, 90)
Shape of data_r tensor: (683702, 90)


In [ ]:
nb_words = min(MAX_NB_WORDS, len(word_index))

embedding_matrix = np.zeros((nb_words+1, EMBEDDING_DIM))

print(word_index)
for (word, idx) in word_index.items():
    if word in word2vec.vocab and idx < MAX_NB_WORDS:
        embedding_matrix[idx] = word2vec.word_vec(word)
embedding_matrix[1]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



array([-2.25585938e-01, -1.95312500e-02,  9.08203125e-02,  2.37304688e-01,
       -2.92968750e-02,  9.32617188e-02, -5.88378906e-02, -4.10156250e-02,
        5.22460938e-02,  2.00195312e-02, -3.44238281e-02, -2.91015625e-01,
       -2.61718750e-01, -1.90429688e-01,  3.85742188e-02,  8.30078125e-03,
        3.14941406e-02,  7.47070312e-02,  1.14746094e-01,  1.24511719e-02,
       -2.11914062e-01,  1.28906250e-01,  2.55859375e-01,  1.08886719e-01,
       -7.03125000e-02,  1.60156250e-01, -4.00390625e-01, -1.16699219e-01,
       -3.22265625e-02, -6.73828125e-02,  1.55273438e-01,  1.25000000e-01,
       -6.22558594e-02,  3.56445312e-02, -1.79687500e-01,  1.58203125e-01,
       -2.53906250e-01,  1.74804688e-01, -1.18164062e-01, -2.19726562e-03,
        1.49536133e-02, -5.71289062e-02,  2.39257812e-01,  1.36718750e-01,
        1.47460938e-01,  7.08007812e-02,  1.72851562e-01, -2.38281250e-01,
       -5.71289062e-02, -1.04492188e-01,  2.23388672e-02,  8.74023438e-02,
        1.43554688e-01,  

In [ ]:
# Assigning labels to the depressive tweets and random tweets data
labels_d = np.array([1] * DEPRES_NROWS)
labels_r = np.array([0] * RANDOM_NROWS)

# Splitting the arrays into test (60%), validation (20%), and train data (20%)
perm_d = np.random.permutation(len(data_d))
idx_train_d = perm_d[:int(len(data_d)*(TRAIN_SPLIT))]
idx_test_d = perm_d[int(len(data_d)*(TRAIN_SPLIT)):int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT))]
idx_val_d = perm_d[int(len(data_d)*(TRAIN_SPLIT+TEST_SPLIT)):]

perm_r = np.random.permutation(len(data_r))
idx_train_r = perm_r[:int(len(data_r)*(TRAIN_SPLIT))]
idx_test_r = perm_r[int(len(data_r)*(TRAIN_SPLIT)):int(len(data_r)*(TRAIN_SPLIT+TEST_SPLIT))]
idx_val_r = perm_r[int(len(data_r)*(TRAIN_SPLIT+TEST_SPLIT)):]

# Combine depressive tweets and random tweets arrays
data_train = np.concatenate((data_d[idx_train_d], data_r[idx_train_r]))
labels_train = np.concatenate((labels_d[idx_train_d], labels_r[idx_train_r]))
data_test = np.concatenate((data_d[idx_test_d], data_r[idx_test_r]))
labels_test = np.concatenate((labels_d[idx_test_d], labels_r[idx_test_r]))
data_val = np.concatenate((data_d[idx_val_d], data_r[idx_val_r]))
labels_val = np.concatenate((labels_d[idx_val_d], labels_r[idx_val_r]))

# Shuffling
perm_train = np.random.permutation(len(data_train))
data_train = data_train[perm_train]
labels_train = labels_train[perm_train]
perm_test = np.random.permutation(len(data_test))
data_test = data_test[perm_test]
labels_test = labels_test[perm_test]
perm_val = np.random.permutation(len(data_val))
data_val = data_val[perm_val]
labels_val = labels_val[perm_val]



In [ ]:

model = Sequential()
# Embedded layer
model.add(Embedding(len(embedding_matrix), EMBEDDING_DIM, weights=[embedding_matrix], 
                            input_length=MAX_SEQUENCE_LENGTH, trainable=False))
# Convolutional Layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layer
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 90, 300)           114161100 
_________________________________________________________________
conv1d (Conv1D)              (None, 90, 32)            28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 45, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 45, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               399600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=1000)

hist = model.fit(data_train, labels_train, \
        validation_data=(data_val, labels_val), \
        epochs=EPOCHS, batch_size=40, shuffle=True, \
        callbacks=[early_stop])

NameError: ignored

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
labels_pred = model.predict(data_test)
labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(labels_test, labels_pred)
print("Accuracy: %.2f%%" % (accuracy*100))
type(data_test)

In [ ]:
print(classification_report(labels_test, labels_pred))

In [ ]:
model.save('/content/gdrive/MyDrive/Capstone/model/model1')

In [ ]:
class LogReg:
    """
    Class to represent a logistic regression model.
    """

    def __init__(self, l_rate, epochs, n_features):
        """
        Create a new model with certain parameters.

        :param l_rate: Initial learning rate for model.
        :param epoch: Number of epochs to train for.
        :param n_features: Number of features.
        """
        self.l_rate = l_rate
        self.epochs = epochs
        self.coef = [0.0] * n_features
        self.bias = 0.0

    def sigmoid(self, score, threshold=20.0):
        """
        Prevent overflow of exp by capping activation at 20.

        :param score: A real valued number to convert into a number between 0 and 1
        """
        if abs(score) > threshold:
            score = threshold * sign(score)
        activation = exp(score)
        return activation / (1.0 + activation)

    def predict(self, features):
        """
        Given an example's features and the coefficients, predicts the class.

        :param features: List of real valued features for a single training example.

        :return: Returns the predicted class (either 0 or 1).
        """
        value = sum([features[i]*self.coef[i] for i in range(len(features))]) + self.bias
        return self.sigmoid(value)

    def sg_update(self, features, label):
        """
        Computes the update to the weights based on a predicted example.

        :param features: Features to train on.
        :param label: Corresponding label for features.
        """
        yhat = self.predict(features)
        e = label - yhat
        self.bias = self.bias + self.l_rate * e * yhat * (1-yhat)
        for i in range(len(features)):
            self.coef[i] = self.coef[i] + self.l_rate * e * yhat * (1-yhat) * features[i]
        return

    def train(self, X, y):
        """
        Computes logistic regression coefficients using stochastic gradient descent.

        :param X: Features to train on.
        :param y: Corresponding label for each set of features.

        :return: Returns a list of model weight coefficients where coef[0] is the bias.
        """
        for epoch in range(self.epochs):
            for features, label in zip(X, y):
                self.sg_update(features, label)
        return self.bias, self.coef

In [ ]:
def get_accuracy(y_bar, y_pred):
    """
    Computes what percent of the total testing data the model classified correctly.

    :param y_bar: List of ground truth classes for each example.
    :param y_pred: List of model predicted class for each example.

    :return: Returns a real number between 0 and 1 for the model accuracy.
    """
    correct = 0
    for i in range(len(y_bar)):
        if y_bar[i] == y_pred[i]:
            correct += 1
    accuracy = (correct / len(y_bar)) * 100.0
    return accuracy

In [ ]:
logreg = LogReg(LEARNING_RATE, EPOCHS, len(data_train[0]))
bias_logreg, weights_logreg = logreg.train(data_train, labels_train)
y_logistic = [round(logreg.predict(example)) for example in data_test]

In [ ]:
accuracy_logistic = get_accuracy(y_logistic, labels_test)
print('Logistic Regression Accuracy: {:0.3f}'.format(accuracy_logistic))

In [ ]:
test=pd.read_csv('/content/gdrive/MyDrive/Capstone/dataset/created_depression.csv')
test.head()
a=test['Tweet'].astype('string')
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(a)
sequences_d = tokenizer.texts_to_sequences(a)
word_index = tokenizer.word_index
data_d = pad_sequences(sequences_d, maxlen=MAX_SEQUENCE_LENGTH)
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
print(nb_words)
print(word_index)
print(MAX_NB_WORDS)
for (word, idx) in word_index.items():
  if word in word2vec.vocab and idx < MAX_NB_WORDS:
    embedding_matrix[idx-1] = word2vec.word_vec(word)
test = data_d
labels_pred=model.predict(test)
print(labels_pred)



